In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-22,France,119836.0,0.0,21470.0,0.0,0.0,0.0
0,2020-04-22,China,83983.0,0.0,4678.0,0.0,0.0,0.0
0,2020-04-22,Italy,186047.0,0.0,25088.0,0.0,0.0,0.0
0,2020-04-22,Spain,208165.0,0.0,21613.0,0.0,0.0,0.0
0,2020-04-22,United States,854031.0,0.0,47596.0,0.0,0.0,0.0
0,2020-04-22,World,2604785.0,0.0,183944.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-23,France,119836.0,119151.0,21470.0,21340.0,685.0,130.0
0,2020-04-23,China,83983.0,84118.0,4678.0,4636.0,-135.0,42.0
0,2020-04-23,Italy,186047.0,187327.0,25088.0,25085.0,-1280.0,3.0
0,2020-04-23,Spain,208165.0,208389.0,21613.0,21717.0,-224.0,-104.0
0,2020-04-23,United States,854031.0,842629.0,47596.0,46784.0,11402.0,812.0
0,2020-04-23,World,2604785.0,2588310.0,183944.0,182808.0,16475.0,1136.0


In [7]:
## dump dataset for us it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")